<a href="https://colab.research.google.com/github/yeeh-l/systemprogramming/blob/master/SystemPrograming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Messages**

(5/5) 完成完整印出2.2的程式 <br>
(5/6) Loc計算的部份完成了，目前包含RESW、RESB、+(第四版)、BASE的特殊處理方式 <br>
(5/6) 我修好wget的問題了，可以直接抓檔案下來了
<BR> -->原來用raw就能變成原始儲存格式，Github真是神
<BR>(5/6) InputFile 可能要加一個List來儲存最後算出來的結果
<BR>(5/8) OutPut結果儲存在listOut，輸出成txt匯出
<BR>(6/1) 我在OPtable新增一個list用來存6-bit binary的opcode

---

    ch2.2內要寫兩個program，我表示成A、B
    A-> 圖2.1變成圖2.2 (產生目的碼)
    B-> 圖2.2變成圖2.3(執行檔)

    *   A的寫法:(我ㄉ想法 ->求意見 不確定實不實用)

    1. 建立optable，內有兩個list分別存
        SIC/XE 指令集 、 運作碼  (格式(format)) 
    2. 建立input的list儲存輸入
        ->  行號 、變數 、 指令 、 物品 、  註解   //->這是我設的變數名稱，使用圖2.1 每個column對應
        EX:  5 、 COPY 、 START 、 1000  、 COPY FILE---  
    3. 再用指令list去跟optable內的指令集對應出 他的格式(3或4) -> 可推斷出每行記憶體位置要加多少(3或4)
    4. 利用推斷出每行input的記憶體位置，產生目的碼 (我會再用list儲存起來)

---
計算Loc的方式
    
    1.def一個16進位轉16進位字串的函數
    2.清空所有的Loc
    3.進入計算自己那一行的Loc
        1.計算START END
        2.跳過註解
        3.計算其他Ins
            1.跳過註解找到前一行的Loc
            2.如果是+(第四版)再加一
    4.處理有影響到下一的Ins (RESW、RESB)
        1.計算下一行的Loc
        2.跳過註解找到下一行的要存的Loc      

# Main code

## 讀入OPtable

In [ ]:
# get files from github
!wget https://raw.githubusercontent.com/yeeh-l/systemprogramming/master/input01.txt
!wget https://raw.githubusercontent.com/yeeh-l/systemprogramming/master/optable.txt

--2020-06-18 08:13:25--  https://raw.githubusercontent.com/yeeh-l/systemprogramming/master/input01.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 713 [text/plain]
Saving to: ‘input01.txt’

input01.txt         100%[===================>]     713  --.-KB/s    in 0s      

2020-06-18 08:13:25 (34.0 MB/s) - ‘input01.txt’ saved [713/713]

--2020-06-18 08:13:27--  https://raw.githubusercontent.com/yeeh-l/systemprogramming/master/optable.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1119 (1.1K) [text/plain]
Saving to: ‘optable.txt’

opta

In [ ]:
class OPtable:
    def __init__(self):
        self.count = 0
        self.AMlist = [] #Assembly Mnemonic
        self.OPlist = [] #opcode in hex
        self.OPcode = [] #opcode in 6-bit binary
    def addlist(self, am, op):
        self.AMlist.append(am)
        self.OPlist.append(op)

        code = bin(int(op, 16))
        code = code[2:-2].zfill(6)
        self.OPcode.append(code)
        self.count += 1
    def showAM(self):
        for i in range(self.count):
            print(self.AMlist[i])
    def showOP(self):
        for i in range(self.count):
            print(self.OPlist[i], self.OPcode[i], sep="\t")
    def show(self):
        for i in range(self.count):
            print(i, self.AMlist[i], self.OPlist[i], self.OPcode[i], sep="\t")
    def __del__(self):
        self.count = 0
        AMlist = []
        OPlist = []
        OPcode = []

In [ ]:
optable = open('/content/optable.txt')
RL = optable.readline()
table = OPtable()
while RL:
    stri = RL.split(" ")
    table.addlist(stri[0], stri[-1].rstrip())
    RL = optable.readline()

In [ ]:
#table.showAM() 
#table.showOP()
table.show()

## 讀入原始敘述list儲存

<H>組譯器指引指令:

In [ ]:
#assembler directives
ADlist = ['START' , 'END' , 'BYTE' , 'WORD' , 'RESB' , 'RESW']

In [ ]:
class InputFile:
    def __init__(self):
        self.Line = 0
        self.Loc = []
        self.Var = []
        self.Ins = []
        self.Item= []
        self.Comment=[]
    def AddLoc(self,inter):
        self.Loc.append(inter)
    def AddVar(self,string):
        self.Var.append(string)
    def AddIns(self,string):
        self.Ins.append(string)
    def AddItem(self,string):
        self.Item.append(string)
    def AddComment(self,string):
        self.Comment.append(string)
    def Show(self):
        for i in range(self.Line):
            print(i, self.Loc[i], self.Var[i], self.Ins[i], self.Item[i], sep='\t')

In [ ]:
input1 = open("/content/input01.txt")
RF = input1.readline()
inP = InputFile()
line = 0
while RF:
    strin = RF.split()
    line += 1

    #排除首行 START 僅有兩個值
    if inP.Line==0 & len(strin)==2:
        print("Error : 首行僅有兩個值")
        break
    
    boo1 = strin[0] in ADlist
    boo2 = strin[0] in table.AMlist
    
    if boo1 | boo2: 
        if len(strin)==1:
            inP.AddLoc('')
            inP.AddVar('')
            inP.AddIns(strin[0])
            inP.AddItem('')
            inP.Line+=1
        else:
            inP.AddLoc('')
            inP.AddVar('')
            inP.AddIns(strin[0])
            inP.AddItem(strin[1])
            inP.Line+=1
    elif strin[0]==".":
        inP.AddLoc('')
        inP.AddVar(strin[0])
        inP.AddIns('')
        inP.AddItem('')
        inP.Line+=1
    else:
        if strin[0] not in inP.Var :
            inP.AddLoc('')
            inP.AddVar(strin[0])
            inP.AddIns(strin[1])
            inP.AddItem(strin[2])
            inP.Line+=1
            ### 缺SYMTAB ### 
        else:
            print("Error : Duplicate symbol")
            break
    RF = input1.readline()

## 計算 loc

In [ ]:
def IntHex_To_StrHex(InpHex, fill_num=0):
    InpHex = str(InpHex)
    InpHex = InpHex[2:]
    InpHex = InpHex.zfill(fill_num)
    return InpHex

for i in range(inP.Line):
    inP.Loc[i]=''

for i in range(inP.Line):
    
    if inP.Ins[i]=='START':
            Loc = inP.Item[i]
            Loc = Loc.zfill(4)
            inP.Loc[i] = Loc
            inP.Loc[i+1] = Loc
    elif inP.Ins[i]=='':
        pass    #跳過註解
    elif inP.Loc[i]=='':                   #1 Test
        pre_id = 1
        PreLoc = inP.Loc[i-pre_id]
        while PreLoc=='':
            pre_id+=1
            PreLoc = inP.Loc[i-pre_id]
        if inP.Ins[i][0]=='+':
            Loc = hex(int(PreLoc, 16)+4)
        else:
            Loc = hex(int(PreLoc, 16)+3)
        inP.Loc[i] = IntHex_To_StrHex(Loc, 4)
    else:
        pass

    if inP.Ins[i] in ['RESW', 'RESB', 'BYTE']:      #2
        next_id = 1
        next_var = inP.Var[i+next_id]
        next_ins = inP.Ins[i+next_id]
        if (next_var=='') & (next_ins==''):
            print('Error:下行缺少Assembly Mnemonic \nLine:', i, inP.Ins[i])
            break
        while next_var=='.':
            next_id+=1
            next_var = inP.Var[i+next_id]
            next_ins = inP.Ins[i+next_id]
        Loc = inP.Loc[i]
        
        if inP.Ins[i]=='RESW':
            NextLoc = hex(int(Loc, 16)+int(inP.Item[i], 10)*3)
        elif inP.Ins[i]=='RESB':
            NextLoc = hex(int(Loc, 16)+int(inP.Item[i], 10))
        elif inP.Ins[i]=='BYTE':
            item = inP.Item[i].split("'")
            if item[0]=='X':
                NextLoc = hex(int(Loc, 16)+1)
            elif item[0]=='C':
                NextLoc = hex(int(Loc, 16)+3)
        else:
            print('Error')
            break
        
        NextLoc = IntHex_To_StrHex(NextLoc, 4)
        inP.Loc[i+next_id] = NextLoc
    else:
        pass

last_loc = inP.Loc[-1]
inP.Loc[-1] = ''

In [ ]:
inP.Show()

0	1000	COPY	START	1000
1	1000	FIRST	STL	RETADR
2	1003	CLOOP	JSUB	RDREC
3	1006		LDA	LENGTH
4	1009		COMP	ZERO
5	100c		JEQ	ENDFIL
6	100f		JSUB	WRREC
7	1012		J	CLOOP
8	1015	ENDFIL	LDA	EOF
9	1018		STA	BUFFER
10	101b		LDA	THREE
11	101e		STA	LENGTH
12	1021		JSUB	WRREC
13	1024		LDL	RETADR
14	1027		RSUB	
15	102a	EOF	BYTE	C'EOF'
16	102d	THREE	WORD	3
17	1030	ZERO	WORD	0
18	1033	RETADR	RESW	1
19	1036	LENGTH	RESW	1
20	1039	BUFFER	RESB	4096
21		.		
22		.		
23		.		
24	2039	RDREC	LDX	ZERO
25	203c		LDA	ZERO
26	203f	RLOOP	TD	INPUT
27	2042		JEQ	RLOOP
28	2045		RD	INPUT
29	2048		COMP	ZERO
30	204b		JEQ	EXIT
31	204e		STCH	BUFFER,X
32	2051		TIX	MAXLEN
33	2054		JLT	RLOOP
34	2057	EXIT	STX	LENGTH
35	205a		RSUB	
36	205d	INPUT	BYTE	X'F1'
37	205e	MAXLEN	WORD	4096
38		.		
39		.		
40		.		
41	2061	WRREC	LDX	ZERO
42	2064	WLOOP	TD	OUTPUT
43	2067		JEQ	WLOOP
44	206a		LDCH	BUFFER,X
45	206d		WD	OUTPUT
46	2070		TIX	LENGTH
47	2073		JLT	WLOOP
48	2076		RSUB	
49	2079	OUTPUT	BYTE	X'05'
50			END	FIRST


### Pass1 to txt

In [ ]:
#list save all output datas to listOut
listOut=[]
for x in range(inP.Line):
   listOut.append(str(x) +"\t"+ inP.Loc[x] +"\t"+ inP.Var[x] +"\t"+ inP.Ins[x] +"\t"+ inP.Item[x] +"\n")
for x in listOut:
  print(x)

In [ ]:
#create file name as output01.txt and write on it 
fo = open('/content/output01.txt','x')   #建立 if exist->註解掉這行
fo = open('/content/output01.txt','w')
row = inP.Line
for x in range(row):
  fo.writelines(listOut[x])   
  print(listOut[x])
fo.close()    #file name:output01.txt #size:971  ->correct

## 計算 object code

In [ ]:
class ObjectFile:
    def __init__(self):
        self.count = 0
        self.ObjCode = []
        self.Choose = []
    def AddObj(self,inter):
        self.ObjCode.append(inter)
        self.count += 1
    def SetChoose(self,bol):
        self.Choose.append(bol)
    def Show(self):
        for i in range(self.count):
            print((i+1)*5, self.ObjCode[i], sep="\t")

In [ ]:
ouP = ObjectFile()

for i in range(inP.Line):
    
    if inP.Ins[i] in ADlist:
        if inP.Ins[i]=='BYTE':
            item = inP.Item[i].split("'")
            if item[0]=='X':
                ouP.AddObj(item[1])
                ouP.SetChoose(1)
            elif item[0]=='C':
                s = ""
                for i in range(len(item[1])):
                    H = hex(ord(item[1][i]))
                    s+=str(H[2:])
                ouP.AddObj(s)
                ouP.SetChoose(1)
            else:
                print('Error')
                break
        elif inP.Ins[i]=='WORD':
            item = hex(int(inP.Item[i]))
            item = item[2:].zfill(6)
            ouP.AddObj(item)
            ouP.SetChoose(1)
        else:
            ouP.AddObj('@') #space
            ouP.SetChoose("NOT Choose")
    elif inP.Ins[i]=='':
        ouP.AddObj('comment')  #space
        ouP.SetChoose("NOT Choose")
    elif inP.Ins[i]=='RSUB':
        ouP.AddObj('4C0000')
        ouP.SetChoose(1)
    elif inP.Ins[i] in table.AMlist:
        am = table.AMlist.index(inP.Ins[i])
        op = table.OPlist[am]

        item = inP.Item[i].split(',')
        if len(item)==2:
            var_loc = inP.Var.index(item[0])
            var = inP.Loc[var_loc]
            var = 8000 + int(var)
        else:    
            var_loc = inP.Var.index(inP.Item[i])
            var = inP.Loc[var_loc]
        
        opCode = str(op) + str(var)
        ouP.AddObj(opCode)
        ouP.SetChoose(1)
    else:
        print('Error：Cannot find vaild Ins')
        break

ouP.Show()

5	@
10	141033
15	482039
20	001036
25	281030
30	301015
35	482061
40	3C1003
45	00102a
50	0C1039
55	00102d
60	0C1036
65	482061
70	081033
75	4C0000
80	454f46
85	000003
90	000000
95	@
100	@
105	@
110	comment
115	comment
120	comment
125	041030
130	001030
135	E0205d
140	30203f
145	D8205d
150	281030
155	302057
160	549039
165	2C205e
170	38203f
175	101036
180	4C0000
185	F1
190	001000
195	comment
200	comment
205	comment
210	041030
215	E02079
220	302064
225	509039
230	DC2079
235	2C1036
240	382064
245	4C0000
250	05
255	@


In [ ]:
class TargetFile:
    def __init__(self):
        self.count = 0
        self.TaCode = []
    def AddObj(self,inter):
        self.TaCode.append(inter)
        self.count += 1
    def Show(self):
        for i in range(self.count):
            print((i+1)*5, self.TaCode[i], sep="\t")

def join_string(words):
    result=""
    for word in words:
        result+=word
        #result+="∧"
    return result

In [ ]:
#ta = TargetFile()

length = hex(int(last_loc, 16)-int(inP.Loc[0], 16))
#n = ['H', inP.Var[0].ljust(6), inP.Loc[0].zfill(6), length[2:].zfill(6)]
#head = join_string(n)
#ta.AddObj(head)
print('H'+inP.Var[0].ljust(6)+"."+inP.Loc[0].zfill(6)+"."+length[2:].zfill(6))

cnt = 0
totcot =0
n = ''
for i in range(ouP.count):    
    if(cnt == 0):
      T = inP.Loc[i]
    
    if(ouP.Choose[i] != "NOT Choose"):
      #print("#",ouP.Choose[i],"#")
      n += '.'
      n += str(ouP.ObjCode[i])
      cnt += 1
      totcot += len(ouP.ObjCode[i])
    #print(ouP.ObjCode[i])
    if cnt==10:
        totcot /= 2
        totcot = int(totcot)
        k += cnt 
        #X = hex(int(inP.Loc[totcot], 16)+3-int(inP.Loc[k], 16))
        #print(hex(totcot))
        print('T.'+ T.zfill(6) +"."+ str(hex(totcot))[2:] + n )
        cnt = 0
        totcot = 0
        n = ''
print('E'+ inP.Loc[0].zfill(6))

HCOPY  .001000.00107a
T.001000.1e.141033.482039.001036.281030.301015.482061.3C1003.00102a.0C1039.00102d
T.00101e.1e.0C1036.482061.081033.4C0000.454f46.000003.000000.041030.001030.E0205d
T.002042.1c.30203f.D8205d.281030.302057.549039.2C205e.38203f.101036.4C0000.F1
T.00205e.1c.001000.041030.E02079.302064.509039.DC2079.2C1036.382064.4C0000.05
E001000


#加分

In [ ]:
#### 2.10
ouP = ObjectFile()

for i in range(inP.Line):

    if inP.Ins[i] in ADlist:
        ouP.AddObj('')
    elif inP.Ins[i] in table.AMlist:
        am = table.AMlist.index(inP.Ins[i])
        op = table.OPcode[am]
        
        # Try PC relative
        var_loc = inP.Var.index(inP.Item[i])
        disp = hex(int(inP.Loc[var_loc], 16)-int(inP.Loc[i], 16)-int('3', 16))
        if disp>int('FFF', 16):
            item = inP.Item[i]
        elif item[1]=='#':
            #direct
        else:
            break
    else:
        print('Error：Cannot find vaild Ins')
        break

#Test

In [ ]:
print(inP.Loc[-2], inP.Loc[0])
hex(int(last_loc, 16)+3-int(inP.Loc[0], 16))

207b 1000


'0x107e'

In [ ]:
#test
x = 0x09
y = hex(x+3)
y = str(y)
y[2:].zfill(4)

In [ ]:
#test
class table:
  def __init__(self):
    self.alist = []

  def addlist(self, el):
    self.alist.append(el)


T = table()
T.addlist("c")
T.addlist("d")

for x in T.alist:
  print(x)

d = 'c'
boolen = d in T.alist
print(boolen)

In [ ]:
a = '.'
b = False
print(a == '.')

In [ ]:
a = 4096
print(hex(a))
y = hex(a)
b = 2000
print(hextoint(y)+b)

In [ ]:
a = "a b c d"

for b in a

b = "a b c "
x= a.split()
y= b.split()
if()
print(x[3])
print(y[3])

In [ ]:
arr = ['a','b','c','d']


lab[arr.index('b')]

print(arr.index('b'))

In [ ]:
import sys
print(sys.version)

3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
